# Использование Tool от LlamaIndex вместе с ConversationalAgent
1. Устанавливаем нужные пакеты

In [ ]:
!pip install llama_index wikipedia gigachain openai
!pip uninstall -y langchain

2. Загружаем статью из Википедии для поиска по ней
3. Инициализируем VectorStoreIndex где будут храниться семантические данные по статье

In [2]:
import os

from llama_index import VectorStoreIndex, download_loader

os.environ["OPENAI_API_KEY"] = "<Ваш OpenAI ключ>"

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Московский метрополитен"], auto_suggest=False, lang="ru"
)

index = VectorStoreIndex.from_documents(documents)

4. Инициализируем промпты для работы LLamaIndex + GigaChat

In [8]:
from langchain_community.chat_models import GigaChat
from llama_index import ServiceContext
from llama_index.llms import ChatMessage, LangChainLLM, MessageRole
from llama_index.prompts import ChatPromptTemplate

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="""Ответь на вопрос, только если в контексте есть информация, чтобы ответить на вопрос.
         Если в контексте нет информации отвечай, что не знаешь ответа.
         Не упоминай, что у тебя есть контекст""",
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Используя только контекст и не свои знания ниже дай ответ на вопрос.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Дай ответ исходя из контекста и не из своих знаний на следующий вопрос: {query_str}\n"
            # noqa: E501
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="""Ответь на вопрос, только если в контексте есть информация, чтобы ответить на вопрос.
         Если в контексте нет информации отвечай, что не знаешь ответа.
         Не упоминай, что у тебя есть контекст""",
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "У нас есть возможность улучить оригинальный ответ (если это нужно) "
            "с контекстной информацией ниже\n"
            "------------\n"
            "{context_msg}\n"
            "------------\n"
            "Учитывая новый контекст, улучши ответ, "
            "чтобы лучше ответить на оригинальный вопрос: {query_str}. "
            "Если контекст бесполезен, выведи исходный ответ\n"
            "Исходный ответ: {existing_answer}"
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

5. Создаем QueryEngine в LLamaIndex, который будет общаться с помощью промптов выше, получать ответ по типу Question-Answer

In [ ]:
llm = GigaChat(
    # Тут данные для входа, для этого примера я использовал 70b модель
)

lama_llm = LangChainLLM(llm=llm)
service_context = ServiceContext.from_defaults(llm=lama_llm)

query_engine = index.as_query_engine(
    text_qa_template=text_qa_template,
    refine_template=refine_template,
    service_context=service_context,
)

6. Инициализируем LLamaIndex Tool

In [18]:
from llama_index.langchain_helpers.agents import IndexToolConfig, LlamaIndexTool

tool_config = IndexToolConfig(
    query_engine=query_engine,
    name="Vector Index",
    description="""Описание: используется, когда тебе нужно узнать информацию про метро
    Параметры: строка для поиска (передавай полностью, то что написал пользователь в Question)""",
    tool_kwargs={"return_direct": True},
)

tool = LlamaIndexTool.from_tool_config(tool_config)

In [22]:
import re
from typing import Union

from langchain.agents.conversational.output_parser import ConvoOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException


def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
    text = re.sub(r"Observation:.*", "", text, 0, re.MULTILINE | re.DOTALL)
    if f"{self.ai_prefix}:" in text:
        return AgentFinish(
            {"output": text.split(f"{self.ai_prefix}:")[-1].strip()}, text
        )
    regex = r"Action: (.*?)[\n]*Action Input: (.*)"
    match = re.search(regex, text)
    if not match:
        raise OutputParserException(f"Could not parse LLM output: `{text}`")
    action = match.group(1)
    action_input = match.group(2)
    return AgentAction(action.strip(), action_input.strip(" ").strip('"'), text)


ConvoOutputParser.parse = parse

7. Создаем Conversation агент, который сможет общаться с пользователем и, если поймет что это нужно, производить поиск по документам и генерировать ответь с помощью LLamaIndex

In [23]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
agent_executor = initialize_agent(
    [tool], llm, agent="conversational-react-description", memory=memory, verbose=True
)

In [24]:
agent_executor.run("Привет, ты кто?")



> Entering new AgentExecutor chain...
Thought: Мне нужно использовать функцию? Нет
AI: Привет! Я — GigaChat, русскоязычный виртуальный помощник. Чем я могу вам помочь сегодня?

> Finished chain.


'Привет! Я — GigaChat, русскоязычный виртуальный помощник. Чем я могу вам помочь сегодня?'

In [25]:
agent_executor.run("Расскажи про мосметротур")



> Entering new AgentExecutor chain...
Thought: Мне нужно использовать функцию? Да
Action: Vector Index
Action Input: "Мосметротур"

Observation: Мосметротур — это проект Городского экскурсионного бюро и Московского метрополитена, который предлагает экскурсии по метрополитену. Гиды проходят обучение и получают информацию об истории развития метро, правилах пользования и эксплуатации метрополитена, технике безопасности и порядке работы в случае чрезвычайных происшествий. Программа включает в себя около 15 экскурсий, включая экскурсии по публично доступным станциям.


> Finished chain.


'Мосметротур — это проект Городского экскурсионного бюро и Московского метрополитена, который предлагает экскурсии по метрополитену. Гиды проходят обучение и получают информацию об истории развития метро, правилах пользования и эксплуатации метрополитена, технике безопасности и порядке работы в случае чрезвычайных происшествий. Программа включает в себя около 15 экскурсий, включая экскурсии по публично доступным станциям.'